In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KLTN/FDP_VN_1year_binary_FIN_WEIGHTED_SEN_2010_2023.csv")
df.head()

,Code,Year,X1,X2,X3,X4,X5,X6,X7,X8,...,X12,X13,X14,X15,X16,X17,X18,X19,SEN,Next_year_binary_distress_label
0,SD2,2010,1.986111,0.393715,0.413592,0.064695,0.412698,0.144177,0.067961,0.049908,...,0.582255,0.633803,0.230769,6.293419,6.244167,0.151571,0.349353,0.685714,0.0,0
1,SD1,2010,1.093110,0.079239,0.185185,0.026941,0.294118,0.039620,0.062963,0.019017,...,0.855784,6.760000,0.005111,6.447306,5.598422,0.045959,0.134707,0.994444,0.0,0
2,BBC,2010,1.809783,0.196311,0.189086,0.059289,0.385321,0.276680,0.057107,0.054018,...,0.281950,0.785235,0.093093,6.632002,6.669498,0.528327,0.718050,0.859813,0.0,0
3,SD4,2010,0.937282,-0.035928,-0.056250,0.035928,0.596639,0.141717,0.056250,0.025948,...,0.762475,-7.611111,0.356877,6.216606,5.768321,0.441118,0.237525,0.751309,0.0,0
4,SCR,2010,2.259175,0.438298,2.979409,0.077571,0.027879,0.008034,0.527305,0.055578,...,0.711313,0.728365,0.462067,8.934982,7.018402,0.001185,0.288160,0.489354,0.0,0


# Test Tunning

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load graph mới
data = torch.load("/content/drive/MyDrive/KLTN/graph_data.pt")
data = data.to(device)

print(data)
print("Num nodes:", data.num_nodes)
print("Num edges:", data.edge_index.shape[1])

# Mapping comp-sector

In [ ]:
unique_company_codes = df['Code'].unique()
print("Unique Company Codes:")
print(unique_company_codes)
print(f"Number of unique company codes: {len(unique_company_codes)}")

Unique Company Codes:
['SD2' 'SD1' 'BBC' ... 'THM' 'PBT' 'HIO']
Number of unique company codes: 1429


In [ ]:
unique_company_codes

array(['SD2', 'SD1', 'BBC', ..., 'THM', 'PBT', 'HIO'], dtype=object)

In [ ]:
unique_codes_df = pd.DataFrame(unique_company_codes, columns=['Company_Code'])
unique_codes_df.to_csv('unique_company_codes.csv', index=False)
print("Unique company codes saved to 'unique_company_codes.csv'")

Unique company codes saved to 'unique_company_codes.csv'


In [ ]:
unique_codes_df.head()

,Company_Code
0,SD2
1,SD1
2,BBC
3,SD4
4,SCR


In [ ]:
df_unique = pd.read_csv("/content/drive/MyDrive/KLTN/unique_company_codes.csv")
df_info   = pd.read_csv("/content/drive/MyDrive/KLTN/danh_sach_cp_24hmoney.csv")

In [ ]:
df_unique.head()

,Company_Code
0,SD2
1,SD1
2,BBC
3,SD4
4,SCR


In [ ]:
df_info.head()

,Mã,Công ty,Ngành,Sàn
0,A32,Công ty cổ phần 32,Giầy dép,UPCOM
1,AAA,Công ty cổ phần Nhựa An Phát Xanh,"Nhựa, cao su & sợi",HOSE
2,AAH,Công ty Cổ phần Hợp Nhất,Khai thác Than,UPCOM
3,AAM,Công ty Cổ phần Thủy sản Mekong,Nuôi trồng nông & hải sản,HOSE
4,AAS,Công ty Cổ phần Chứng Khoán Smart Invest,Môi giới chứng khoán,UPCOM


In [ ]:
import difflib

print("Columns in unique:", df_unique.columns.tolist())
print("Columns in info:", df_info.columns.tolist())

Columns in unique: ['Company_Code']
Columns in info: ['Mã', 'Công ty', 'Ngành', 'Sàn']


In [ ]:
df_info = df_info.rename(columns={"Mã": "ticker"})
df_info = df_info.rename(columns={"Công ty": "company_name"})
df_info = df_info.rename(columns={"Ngành": "sector"})

df_unique['Company_Code'] = df_unique['Company_Code'].str.strip().str.upper()
df_info['ticker']         = df_info['ticker'].str.strip().str.upper()

df_merged = df_unique.merge(
    df_info[['ticker', 'company_name', 'sector']],
    left_on='Company_Code',
    right_on='ticker',
    how='left'
)

print(df_merged.head(20))
print("Shape:", df_merged.shape)

   Company_Code ticker                                   company_name  \
0           SD2    SD2                      Công ty Cổ phần Sông Đà 2   
1           SD1    SD1                      Công ty Cổ phần Sông Đà 1   
2           BBC    BBC                         Công ty Cổ phần Bibica   
3           SD4    SD4                      Công ty Cổ phần Sông Đà 4   
4           SCR    SCR      Công ty Cổ phần Địa ốc Sài Gòn Thương Tín   
5           SD3    SD3                      Công ty Cổ phần Sông Đà 3   
6           SD5    SD5                      Công ty Cổ phần Sông Đà 5   
7           HNM    HNM                     Công ty Cổ phần Sữa Hà Nội   
8           SD8    SD8                      Công ty Cổ phần Sông Đà 8   
9           SD7    SD7                      Công ty Cổ phần Sông Đà 7   
10          SD9    SD9                      Công ty Cổ phần Sông Đà 9   
11          HNP    HNP                 Công ty cổ phần Hanel Xốp nhựa   
12          HNR    HNR  Công ty Cổ phần Rượu và Nướ

In [ ]:
df_merged.head()

,Company_Code,ticker,company_name,sector
0,SD2,SD2,Công ty Cổ phần Sông Đà 2,Xây dựng
1,SD1,SD1,Công ty Cổ phần Sông Đà 1,Xây dựng
2,BBC,BBC,Công ty Cổ phần Bibica,Thực phẩm
3,SD4,SD4,Công ty Cổ phần Sông Đà 4,Xây dựng
4,SCR,SCR,Công ty Cổ phần Địa ốc Sài Gòn Thương Tín,Bất động sản


In [ ]:
cate = df_merged['sector'].unique()
cate = pd.DataFrame(cate)
cate

,0
0,Xây dựng
1,Thực phẩm
2,Bất động sản
3,"Nhựa, cao su & sợi"
4,Vang & Rượu mạnh
...,...
73,Tài chính đặc biệt
74,Đồ chơi
75,Khai thác vàng
76,Đường sắt


In [ ]:
null_sector_rows = df_merged[df_merged['sector'].isnull()]
print("Rows with null sector:")
display(null_sector_rows)

Rows with null sector:


,Company_Code,ticker,company_name,sector
18,SDJ,NaN,NaN,NaN
42,BDB,NaN,NaN,NaN
114,V11,NaN,NaN,NaN
173,KAC,NaN,NaN,NaN
197,KCE,NaN,NaN,NaN
220,BST,NaN,NaN,NaN
322,MTC,NaN,NaN,NaN
337,TBT,NaN,NaN,NaN
356,ING,NaN,NaN,NaN
369,CC4,NaN,NaN,NaN


In [ ]:
manual_df = pd.read_csv("/content/drive/MyDrive/KLTN/manual_label_sector.csv")
manual_df.head()

,Company_Code,ticker,company_name,sector
0,SDJ,SDJ,Công ty Cổ phần Sông Đà,Vật liệu xây dựng & Nội thất
1,BDB,BDB,CTCP Sách và Thiết bị Bình Định,Dịch vụ truyền thông
2,V11,V11,CTCP Xây dựng Số 11,Xây dựng
3,KAC,KAC,CTCP Đầu tư Địa ốc Khang An,Bất động sản
4,KCE,KCE,CTCP Bê tông Ly tâm Điện Lực Khánh Hòa,Vật liệu xây dựng & Nội thất


In [ ]:
df_tmp = df_merged.merge(manual_df, on='Company_Code', how='left', suffixes=('', '_new'))

for col in ['ticker', 'company_name', 'sector']:
    df_tmp[col] = df_tmp[col].fillna(df_tmp[col + '_new'])

df_tmp = df_tmp.drop(columns=[c for c in df_tmp.columns if c.endswith('_new')])


df_tmp.count()

,0
Company_Code,1429
ticker,1429
company_name,1429
sector,1429


In [ ]:
df_merged.head()

,Company_Code,ticker,company_name,sector
0,SD2,SD2,Công ty Cổ phần Sông Đà 2,Xây dựng
1,SD1,SD1,Công ty Cổ phần Sông Đà 1,Xây dựng
2,BBC,BBC,Công ty Cổ phần Bibica,Thực phẩm
3,SD4,SD4,Công ty Cổ phần Sông Đà 4,Xây dựng
4,SCR,SCR,Công ty Cổ phần Địa ốc Sài Gòn Thương Tín,Bất động sản


In [ ]:
df_merged[df_merged['sector']=='NaN']

,Company_Code,ticker,company_name,sector


In [ ]:
df_merged[df_merged['sector'].isnull()]

,Company_Code,ticker,company_name,sector
18,SDJ,NaN,NaN,NaN
42,BDB,NaN,NaN,NaN
114,V11,NaN,NaN,NaN
173,KAC,NaN,NaN,NaN
197,KCE,NaN,NaN,NaN
220,BST,NaN,NaN,NaN
322,MTC,NaN,NaN,NaN
337,TBT,NaN,NaN,NaN
356,ING,NaN,NaN,NaN
369,CC4,NaN,NaN,NaN


In [ ]:
df_merged.to_csv("unique_company_with_sector.csv", index=False)
print("Saved to unique_company_with_sector.csv")

Saved to unique_company_with_sector.csv


# Create Graph

## Create Node List

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KLTN/FDP_VN_1year_binary_FIN_WEIGHTED_SEN_2010_2023.csv")

df['Code'] = df['Code'].astype(str).str.strip().str.upper()

df = df.sort_values(['Code', 'Year']).reset_index(drop=True)

df['node_id'] = df.groupby(['Code', 'Year']).ngroup()

print("Số lượng nodes (firm-year observations):", df['node_id'].nunique())
df[['node_id','Code','Year']].head(15)

Số lượng nodes (firm-year observations): 12678


,node_id,Code,Year
0,0,A32,2016
1,1,A32,2017
2,2,A32,2018
3,3,A32,2019
4,4,AAA,2010
5,5,AAA,2011
6,6,AAA,2012
7,7,AAA,2013
8,8,AAA,2014
9,9,AAA,2015


### Merge Ngành

In [ ]:
import itertools
from tqdm import tqdm

In [ ]:
df.head()

,Code,Year,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,SEN,Next_year_binary_distress_label,node_id
0,A32,2016,1.147157,0.094624,0.069952,0.092473,0.404908,0.141935,0.068362,0.083871,...,2.750000,0.005831,6.142037,6.444131,0.234409,0.350538,0.990066,0.387927,0,0
1,A32,2017,1.161491,0.103792,0.084967,0.083832,0.375000,0.131737,0.068627,0.079840,...,2.500000,0.005348,6.216606,6.416732,0.217565,0.351297,0.990769,0.489708,0,1
2,A32,2018,1.263158,0.149254,0.108359,0.110874,0.308458,0.132196,0.080495,0.110874,...,2.342857,0.005952,6.150603,6.470800,0.260128,0.428571,0.992537,0.515522,0,2
3,A32,2019,1.423810,0.204598,0.128799,0.131034,0.366071,0.188506,0.082489,0.137931,...,2.044944,0.003344,6.075346,6.538140,0.285057,0.514943,0.995261,0.499329,0,3
4,AAA,2010,1.338028,0.111628,0.106038,0.155039,0.578947,0.289922,0.147275,0.172093,...,1.125000,0.378947,6.469250,6.520621,0.533333,0.500775,0.663551,0.000000,0,4


In [ ]:
df_sector = pd.read_csv("/content/drive/MyDrive/KLTN/unique_company_with_sector (1).csv")

df_sector['Company_Code'] = df_sector['Company_Code'].astype(str).str.strip().str.upper()

# MERGE FINANCIAL DATA + SECTOR
df_merged = df.merge(
    df_sector[['Company_Code','sector']],
    left_on='Code',
    right_on='Company_Code',
    how='left'
)

print(df_merged[['Code','Year','sector']].head())
print("Số dòng:", len(df_merged))
print("Số missing sector:", df_merged['sector'].isna().sum())

  Code  Year              sector
0  A32  2016            Giầy dép
1  A32  2017            Giầy dép
2  A32  2018            Giầy dép
3  A32  2019            Giầy dép
4  AAA  2010  Nhựa, cao su & sợi
Số dòng: 12678
Số missing sector: 0


In [ ]:
df_merged = df_merged.drop(columns=['Company_Code'])
df_merged.head()

,Code,Year,X1,X2,X3,X4,X5,X6,X7,X8,...,X14,X15,X16,X17,X18,X19,SEN,Next_year_binary_distress_label,node_id,sector
0,A32,2016,1.147157,0.094624,0.069952,0.092473,0.404908,0.141935,0.068362,0.083871,...,0.005831,6.142037,6.444131,0.234409,0.350538,0.990066,0.387927,0,0,Giầy dép
1,A32,2017,1.161491,0.103792,0.084967,0.083832,0.375000,0.131737,0.068627,0.079840,...,0.005348,6.216606,6.416732,0.217565,0.351297,0.990769,0.489708,0,1,Giầy dép
2,A32,2018,1.263158,0.149254,0.108359,0.110874,0.308458,0.132196,0.080495,0.110874,...,0.005952,6.150603,6.470800,0.260128,0.428571,0.992537,0.515522,0,2,Giầy dép
3,A32,2019,1.423810,0.204598,0.128799,0.131034,0.366071,0.188506,0.082489,0.137931,...,0.003344,6.075346,6.538140,0.285057,0.514943,0.995261,0.499329,0,3,Giầy dép
4,AAA,2010,1.338028,0.111628,0.106038,0.155039,0.578947,0.289922,0.147275,0.172093,...,0.378947,6.469250,6.520621,0.533333,0.500775,0.663551,0.000000,0,4,"Nhựa, cao su & sợi"


In [ ]:
df = df_merged
df.head()

,Code,Year,X1,X2,X3,X4,X5,X6,X7,X8,...,X14,X15,X16,X17,X18,X19,SEN,Next_year_binary_distress_label,node_id,sector
0,A32,2016,1.147157,0.094624,0.069952,0.092473,0.404908,0.141935,0.068362,0.083871,...,0.005831,6.142037,6.444131,0.234409,0.350538,0.990066,0.387927,0,0,Giầy dép
1,A32,2017,1.161491,0.103792,0.084967,0.083832,0.375000,0.131737,0.068627,0.079840,...,0.005348,6.216606,6.416732,0.217565,0.351297,0.990769,0.489708,0,1,Giầy dép
2,A32,2018,1.263158,0.149254,0.108359,0.110874,0.308458,0.132196,0.080495,0.110874,...,0.005952,6.150603,6.470800,0.260128,0.428571,0.992537,0.515522,0,2,Giầy dép
3,A32,2019,1.423810,0.204598,0.128799,0.131034,0.366071,0.188506,0.082489,0.137931,...,0.003344,6.075346,6.538140,0.285057,0.514943,0.995261,0.499329,0,3,Giầy dép
4,AAA,2010,1.338028,0.111628,0.106038,0.155039,0.578947,0.289922,0.147275,0.172093,...,0.378947,6.469250,6.520621,0.533333,0.500775,0.663551,0.000000,0,4,"Nhựa, cao su & sợi"


In [ ]:
YEAR_COL = 'Year'
SECTOR_COL = 'sector'
NODE_COL = 'node_id'
MAX_GROUP_SIZE = 300
K_WITHIN_LARGE_GROUP = 20

In [ ]:
groups = df.groupby([YEAR_COL, SECTOR_COL])[NODE_COL].apply(list).reset_index(name='nodes')
groups

,Year,sector,nodes
0,2010,Bán lẻ phức hợp,"[1026, 1763, 4145, 7492, 10267]"
1,2010,Bất động sản,"[874, 1123, 1319, 1585, 1614, 1817, 2124, 2136..."
2,2010,Chuyển phát nhanh,[12433]
3,2010,Chăm sóc y tế,[10863]
4,2010,Chất thải & Môi trường,"[6418, 9099]"
...,...,...,...
939,2022,Đường sắt,"[4619, 9673]"
940,2022,Đồ chơi,[6763]
941,2022,Đồ gia dụng lâu bền,[9305]
942,2022,Đồ gia dụng một lần,[10427]


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, BatchNorm, GATv2Conv
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
import scipy.sparse as sp

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KLTN/FDP_VN_1year_binary_FIN_WEIGHTED_SEN_2010_2023.csv")
df['Code'] = df['Code'].astype(str).str.strip().str.upper()
df = df.sort_values(['Code', 'Year']).reset_index(drop=True)

df_sector_raw = pd.read_csv("/content/drive/MyDrive/KLTN/unique_company_with_sector (1).csv")
df_sector_raw['ticker'] = df_sector_raw['ticker'].astype(str).str.strip().str.upper()
df = df.merge(df_sector_raw[['ticker', 'sector']], left_on='Code', right_on='ticker', how='left')
df['sector'] = df['sector'].fillna('Unknown')

feature_cols = [f'X{i}' for i in range(1, 20)] + ['SEN']
X = df[feature_cols].values
y = df['Next_year_binary_distress_label'].values

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Mask Train (2010-2021) / Test (2022)
train_indices = df[df['Year'] <= 2021].index
test_indices = df[df['Year'] == 2022].index

train_mask = torch.zeros(len(df), dtype=torch.bool)
test_mask = torch.zeros(len(df), dtype=torch.bool)
train_mask[train_indices] = True
test_mask[test_indices] = True

In [ ]:
k_global = 5
A_global = kneighbors_graph(X_norm, k_global, mode='connectivity', include_self=False)

rows_list = []
cols_list = []
k_sector = 10

groups = df.groupby(['Year', 'sector'])
for (year, sector), group_idxs in groups:
    idxs = group_idxs.index.values
    if len(idxs) > k_sector + 1:

        X_sub = X_norm[idxs]
        A_sub = kneighbors_graph(X_sub, k_sector, mode='connectivity', include_self=False)

        sub_rows, sub_cols = A_sub.nonzero()
        rows_list.append(idxs[sub_rows])
        cols_list.append(idxs[sub_cols])

if rows_list:
    sector_rows = np.concatenate(rows_list)
    sector_cols = np.concatenate(cols_list)

    data_ones = np.ones(len(sector_rows))
    A_sector = sp.coo_matrix((data_ones, (sector_rows, sector_cols)), shape=(len(df), len(df)))
else:
    A_sector = sp.coo_matrix((len(df), len(df)))

A_final = A_global + A_sector
edge_index = torch.tensor(np.array(A_final.nonzero()), dtype=torch.long)
print(f"-> Tổng số cạnh sau khi tinh chỉnh: {edge_index.shape[1]}")

x_tensor = torch.tensor(X_norm, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.long)
data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor)
data.train_mask = train_mask
data.test_mask = test_mask

-> Tổng số cạnh sau khi tinh chỉnh: 161962
